# Calibration Demo

<div class="alert alert-block alert-info">
<p><b>Information:</b> This notebook shows a developed <code>scipp</code> function for calibrating data and compared its output to those from <code>Mantid</code>'s original algorithms.</p>
    
<p>We use the following equation to convert units between <i>tof</i> and <i>d-spacing</i> ($d$): $TOF = DIFC * d + DIFA * d^2 + TZERO$, where $DIFC$, $DIFA$ and $TZERO$ are stored in the calibration file.</p> 
    
<p>We first compare the structure of the calibration data and then the data after converting from time-of-flight to d-spacing taking calibration into account.</p>  
      
<p><b>Requirements:</b> To run this notebook, you need <code>mantid</code>, <code>scipp</code>, <code>scippneutron</code> and <code>numpy</code> installed as well as the Python script <code>calibration.py</code> placed in the same folder as this notebook. </p>
  
The required datafiles are available from Mantid:
<ul>
<li> detectors calibration file <a href="http://198.74.56.37/ftp/external-data/MD5/c181221ebef9fcf30114954268c7a6b6">PG3_FERNS_d4832_2011_08_24.cal</a></li>
<li> sample run <a href="http://198.74.56.37/ftp/external-data/MD5/d5ae38871d0a09a28ae01f85d969de1e">PG3_4844_event.nxs</a></li>
</ul>
Once downloaded the files should be renamed and placed in the same folder as this notebook.
</div>

In [ ]:
import numpy as np
import mantid.simpleapi as mapi
import scipp as sc
import scippneutron as scn

import calibration

sample_file = 'PG3_4844_event.nxs'
calibration_file = 'PG3_FERNS_d4832_2011_08_24.cal'

## Mantid
### Load sample and calibration

In [ ]:
sample_mantid = mapi.Load(sample_file)

In [ ]:
mapi.LoadDiffCal(Filename=calibration_file, 
                 InstrumentFilename='POWGEN_Definition_2011-02-25.xml',
                 WorkspaceName='PG3')

### Characteristics of the `cal` tableworkspace

In [ ]:
print(f"Number of columns: {mapi.mtd['PG3_cal'].columnCount()}")  # 4
print(f"Column names: {mapi.mtd['PG3_cal'].getColumnNames()}") # ['detid', 'difc', 'difa', 'tzero']
print(f"Column types: {mapi.mtd['PG3_cal'].columnTypes()}")  # ['int', 'double', 'double', 'double']
print(f"Number of rows: {mapi.mtd['PG3_cal'].rowCount()}") # 24794

print(f"First and last values of {mapi.mtd['PG3_cal'].getColumnNames()[0]}: {mapi.mtd['PG3_cal'].column(0)[0]}, {mapi.mtd['PG3_cal'].column(0)[-1]}")
print(f"First and last values of {mapi.mtd['PG3_cal'].getColumnNames()[1]}: {mapi.mtd['PG3_cal'].column(1)[0]}, {mapi.mtd['PG3_cal'].column(1)[-1]}")
print(f"First and last values of {mapi.mtd['PG3_cal'].getColumnNames()[2]}: {mapi.mtd['PG3_cal'].column(2)[0]}, {mapi.mtd['PG3_cal'].column(2)[-1]}")
print(f"First and last values of {mapi.mtd['PG3_cal'].getColumnNames()[3]}: {mapi.mtd['PG3_cal'].column(3)[0]}, {mapi.mtd['PG3_cal'].column(3)[-1]}")

### Convert taking calibration into account

In [ ]:
mapi.AlignDetectors(InputWorkspace=sample_mantid,
                    OutputWorkspace='PG3_4844_cal',
                    CalibrationWorkspace='PG3_cal')

In [ ]:
print((f"Spectrum numbers stored in calibration (first, last):"
      f"{mapi.mtd['PG3_4844_cal'].getSpectrumNumbers()[0]} "
      f"{mapi.mtd['PG3_4844_cal'].getSpectrumNumbers()[-1]}"))

In [ ]:
print('------------Input workspace-----------')
print(f"Unit for x-axis: {mapi.mtd['sample_mantid'].getXDimension().name}") # Time-of_flight
print(f"Unit for y-axis: {mapi.mtd['sample_mantid'].getYDimension().name}") # Spectrum
print(f"Number of events: {mapi.mtd['sample_mantid'].getNumberEvents()}") # 17926980
print(f"Number of histograms: {mapi.mtd['sample_mantid'].getNumberHistograms()}") # 24794

print((f"First and last values of x-axis: {mapi.mtd['sample_mantid'].readX(0)[0]} "
       f"{mapi.mtd['sample_mantid'].readX(0)[-1]}"))

print('------------Output workspace------------')
print(f"Unit for x-axis: {mapi.mtd['PG3_4844_cal'].getXDimension().name}") # d-Spacing
print(f"Unit for y-axis: {mapi.mtd['PG3_4844_cal'].getYDimension().name}") # Spectrum
print(f"Number of events: {mapi.mtd['PG3_4844_cal'].getNumberEvents()}") # 17926980
print(f"Number of histograms: {mapi.mtd['PG3_4844_cal'].getNumberHistograms()}") # 24794

print((f"First and last values of x-axis: {mapi.mtd['PG3_4844_cal'].readX(0)[0]} "
       f"{mapi.mtd['PG3_4844_cal'].readX(0)[-1]}"))

In [ ]:
# Compare y-values of input and output workspaces for a spectrum.
# they should be the same since only the x-axis is modified (tof -> d-spacing)
spectrum_index = 550
print((f"Are the count values the same for the input and output of AlignDetectors "
       f"for spectrum {spectrum_index}?"))
all([mapi.mtd['PG3_4844_cal'].readY(spectrum_index)[i]==mapi.mtd['sample_mantid'].readY(spectrum_index)[i]] 
    for i in range(len(mapi.mtd['PG3_4844_cal'].readY(spectrum_index))))

### In scipp
#### Load sample and calibration

In [ ]:
sample = scn.load(sample_file,
                  advanced_geometry=True,
                  load_pulse_times=False,
                  mantid_args={'LoadMonitors': True})
sample

In [ ]:
cal = calibration.load_calibration(calibration_file,
                                   mantid_args={'InstrumentFilename': 'POWGEN_Definition_2011-02-25.xml'})

In [ ]:
cal

#### Characteristics of the `cal` dataset

The output of `calibration.load_calibration` in `scipp` is a dataset, which contains `group`, `mask` and the values of `difa`, `difc` and `tzero` used to convert time-of-flight to d-spacing. In `Mantid`, 3 workspaces are generated 

In [ ]:
set(cal.keys()) == set(['mask', 'difc', 'tzero', 'difa', 'group'])

In [ ]:
print(f"Number of rows: {len(cal['difa'].values)}") # 24794
print((f"First and last values of detectors' ids: {cal.coords['detector'].values[0]}, "
      f"{cal.coords['detector'].values[-1]}"))
print(f"First and last values of difc: {cal['difc'].values[0]}, {cal['difc'].values[-1]}")
print(f"First and last values of difa: {cal['difa'].values[0]}, {cal['difa'].values[-1]}")
print(f"First and last values of tzero: {cal['tzero'].values[0]}, {cal['tzero'].values[0-1]}")

#### Convert with calibration

In [ ]:
sample_dspacing = calibration.convert_with_calibration(sample, cal)
sample_dspacing

In [ ]:
print('------------Input dataset-----------')
print(f"Units for axes: {sample.dims}") # spectrum, tof
print(f"Number of events: {np.sum(sample.bins.constituents['data'].values):.0f}") # 17926980
print(f"Number of spectra: {len(sample.coords['spectrum'].values)}") # 24794

print((f"First and last values of x-axis: {sample.coords['tof'].values[0]} "
       f"{sample.coords['tof'].values[-1]}"))

print('------------Output dataset------------')
print(f"Unit for axes: {sample_dspacing.dims}") # spectrum, d-Spacing
print(f"Number of events: {np.sum(sample_dspacing.bins.constituents['data'].values):.0f}") # 17926980
print(f"Number of spectra: {len(sample_dspacing.coords['spectrum'].values)}") # 24794

print((f"First and last values of x-axis: {sample_dspacing.coords['dspacing'].values[0]} "
      f"{sample_dspacing.coords['dspacing'].values[-1]}"))

### Converting manually
The formula to use is `d-spacing` = (`tof` - `tzero`) / `difc` since `difa=0` in the calibration file used for testing.  
The values of `tzero` and `difc` are read from the calibration file.

In [ ]:
# We check that the values of difa are all equal to 0
all([item==0 for item in cal['difa'].values])

In [ ]:
# we express 1/difc in Angstrom / microseconds
mult_difc = sc.to_unit(1 * sc.units.one/cal["difc"].data, unit='angstrom/us')
mult_difc

In [ ]:
# check if coefficient contains any NaN
np.any(np.isnan(mult_difc.values))

In [ ]:
# Convert tof axis to spacing
tof2dspacing = (sample.coords['tof'] - cal["tzero"].data) / cal["difc"].data
tof2dspacing

In [ ]:
# Comparing x-axis (d-spacing) from manual conversion and output of convert_with_calibration
all(tof2dspacing.values[0,:] == sample_dspacing.coords['dspacing'].values[:,0]) & all(tof2dspacing.values[1,:] == sample_dspacing.coords['dspacing'].values[:,1])